In [1]:
import torch
import torchvision

class TrainData:
    def __init__(self):
        self._download_train_data()
        
    def _download_train_data(self):
        orig_train = torchvision.datasets.CIFAR10('data', train=True, transform=torchvision.transforms.ToTensor(), download=True)

        classes = tuple(orig_train.classes)

        keep_labels = (classes.index('cat'), classes.index('dog'))  #tuple(range(len(classes)))
        aux_labels = tuple(classes.index(a) for a in ['airplane', 'automobile', 'bird', 'ship', 'truck'])
        n = 5000 * len(keep_labels)
        auxn = 25000
        channels = 3
        w = 32
        h = 32
        X = torch.zeros((n, channels, w, h))
        y = torch.zeros((n,), dtype=torch.long)
        auxX = torch.zeros((auxn, channels, w, h))

        j = 0
        auxj = 0
        for x,label in orig_train:
            if label in keep_labels:
                X[j,:,:,:] = x
                y[j] = keep_labels.index(label)
                j += 1
            if label in aux_labels:
                auxX[auxj,:,:,:] = x
                auxj += 1
        if j != n:
            raise Exception(f"Wrong number of valid examples {j} {n}")
        if auxj != auxn:
            raise Exception(f"Wrong number of aux examples {auxj} {auxn}")
        self._X = X
        self._y = y
        self._n = n
        self._auxX = auxX
        self._auxn = auxn
        self._n_classes = len(keep_labels)
        
    def in_distribution_dataset(self):
        return [(self._X[i], self._y[i]) for i in range(self._n)]
        
    def mixed_dataset(self):
        ind = [(self._X[i], (self._y[i], 0)) for i in range(self._n)]
        ood = [(self._auxX[i], (0.5, 1)) for i in range(self._auxn)]
        return ind + ood
    
    def n_classes(self):
        return self._n_classes
        
train_data = TrainData()
print("Have training data")

Files already downloaded and verified
Have training data


In [2]:

class TestData:
    def __init__(self):
        self._download_test_data()
        
    def _download_test_data(self):
        orig_train = torchvision.datasets.CIFAR10('data', train=False, transform=torchvision.transforms.ToTensor(), download=True)

        classes = tuple(orig_train.classes)

        keep_labels = (classes.index('cat'), classes.index('dog'))   # tuple(range(len(classes)))
        ood_labels = tuple(classes.index(a) for a in ['deer','frog','horse'])
        n = 1000 * len(keep_labels)
        oodn = 3000
        channels = 3
        w = 32
        h = 32
        X = torch.zeros((n, channels, w, h))
        y = torch.zeros((n,), dtype=torch.long)
        oodX = torch.zeros((oodn, channels, w, h))

        j = 0
        oodj = 0
        for x,label in orig_train:
            if label in keep_labels:
                X[j,:,:,:] = x
                y[j] = keep_labels.index(label)
                j += 1
            if label in ood_labels:
                oodX[oodj,:,:,:] = x
                oodj += 1
        if j != n:
            raise Exception(f"Wrong number of valid examples {j} {n}")
        if oodj != oodn:
            raise Exception(f"Wrong number of ood examples {oodj} {oodn}")
        self._X = X
        self._y = y
        self._n = n
        self._oodX = oodX
        self._oodn = oodn
        
    def in_distribution_dataset(self):
        return [(self._X[i], self._y[i]) for i in range(self._n)]
        
#    def mixed_dataset(self):
#        ind = [(self._X[i], (self._y[i], 0)) for i in range(self._n)]
#        ood = [(self._auxX[i], (0.5, 1)) for i in range(self._auxn)]
#        return ind + ood
        
test_data = TestData()
print("Have test data")

Files already downloaded and verified
Have test data


In [3]:
class SimpleCnnModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Conv2d(3, 32, 3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Conv2d(32, 64, 3),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(2, 2),
            torch.nn.Conv2d(64, 32, 4),
            torch.nn.ReLU(),
            torch.nn.Flatten(),
            torch.nn.Linear(288, 1),
            torch.nn.Sigmoid(),
            #torch.nn.ReLU(),
            #torch.nn.Linear(64, n_classes),
        )

    def forward(self, x):
        return self.layers(x).flatten(start_dim=0)


In [20]:
class Experiment:
    def __init__(self, num_epochs=100):
        global train_data
        global test_data
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        if train_data.n_classes() != 2:
            raise Exception("Binary classifiers only")
        self.model = SimpleCnnModel().to(self.device)
        self.dataloader = torch.utils.data.DataLoader(
            train_data.in_distribution_dataset(),
            batch_size=64,
            shuffle=True
        )
        self.test_dataloader = torch.utils.data.DataLoader(
            test_data.in_distribution_dataset(),
            batch_size=64,
            shuffle=False
        )
        self.loss_fn = torch.nn.BCELoss(reduction='sum')
        self.optimizer = torch.optim.Adam(self.model.parameters(), weight_decay=0.4, lr=0.001)
        self.num_epochs = num_epochs

    def _train(self):
        print(self.device)
        train_accs = []
        test_accs = []
        epochs = []
        for epoch in range(self.num_epochs):
            #print(f"=== Epoch {epoch}===")
            running_loss = torch.zeros(())
            train_accuracy = torch.zeros(())
            running_count = 0
            for inputs, labels in self.dataloader:
                self.optimizer.zero_grad()
                outputs = self.model(inputs.to(self.device))
                predictions = outputs.detach() >= 0.5
                loss = self.loss_fn(outputs, labels.to(self.device).to(torch.float))
                loss.backward()
                self.optimizer.step()
                running_loss += loss.detach().to('cpu')
                train_accuracy += (predictions.to('cpu') == labels).sum()
                running_count += outputs.shape[0]
                    
            test_accuracy = torch.zeros(())
            test_count = 0
            with torch.no_grad():
                for inputs, labels in self.test_dataloader:
                    outputs = self.model(inputs.to(self.device))
                    predictions = outputs >= 0.5
                    test_accuracy += (predictions.to('cpu') == labels).sum()
                    test_count += outputs.shape[0]
            train_acc = train_accuracy.item() / running_count
            test_acc = test_accuracy.item() / test_count
            train_accs.append(train_acc)
            test_accs.append(test_acc)
            epochs.append(epoch)
            if epoch % 5 == 4:
                print('Epoch', f'{epoch:2}',
                      'Train_acc', f'{train_acc:10}',
                      'Test_acc', test_acc)
        return epochs, train_accs, test_accs

In [ ]:
import matplotlib.pyplot as plt
e, tr, te = Experiment()._train()
plt.rcParams['figure.figsize'] = [8, 8]
plt.plot(e, tr, label='Train')
plt.plot(e, te, label='Test')
plt.legend()
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
None

cuda:0
Epoch  4 Train_acc     0.5912 Test_acc 0.577
Epoch  9 Train_acc     0.6087 Test_acc 0.604
Epoch 14 Train_acc     0.6248 Test_acc 0.6225
Epoch 19 Train_acc      0.637 Test_acc 0.6275
Epoch 24 Train_acc     0.6428 Test_acc 0.6325
Epoch 29 Train_acc     0.6598 Test_acc 0.6495
Epoch 34 Train_acc     0.6738 Test_acc 0.6345
Epoch 39 Train_acc     0.6842 Test_acc 0.668
Epoch 44 Train_acc     0.6862 Test_acc 0.673
Epoch 49 Train_acc      0.699 Test_acc 0.684
